In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from scombi_do.app import scombi
from helpers import item_to_img_overlay, my_read_method
from pystac import STAC_IO, read_file, Item, Catalog, CatalogType
from shapely.geometry import shape
from ipyleaflet import Map

### Stage the STAC catalogs

In [ ]:
input_references = ['https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414.json', 
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.json',
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json',
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/48/M/YT/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429.json']

In [ ]:
STAC_IO.read_text_method = my_read_method
    
catalogs = []

for index, input_reference in enumerate(input_references):

    items = []
    
    thing = read_file(input_reference)

    if isinstance(thing, Item):

        items.append(thing)

    elif isinstance(thing, Catalog):

        for item in thing.get_items():

            items.append(item)

    # create catalog
    catalog = Catalog(id=items[0].id,
              description='staged STAC catalog with {}'.format(items[0].id))

    catalog.add_items(items)

    catalog.normalize_and_save(root_href=items[0].id,
                               catalog_type=CatalogType.RELATIVE_PUBLISHED)

    catalog.describe()
    
    catalogs.append(catalog.get_self_href())

In [ ]:
catalogs

### Scombi-do

#### Simple RGB combination

In [ ]:
expressions = ['(interp v1 (asarray 0 10000) (asarray 0 1))', 
               '(interp v2 (asarray 0 10000) (asarray 0 1))',
               '(interp v3 (asarray 0 10000) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [ ]:
bands = ['red', 'green', 'blue']

Define the input local STAC catalogs

In [ ]:
channel_inputs = [catalogs[0], 
                  catalogs[0],
                  catalogs[0]]
channel_inputs

In [ ]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.35' 

In [ ]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [ ]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['color'] = color
params['aoi'] = aoi


In [ ]:
result = scombi(**params)

In [ ]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m

#### Normalized difference

In [ ]:
expressions = ['(interp (/ (- v1 v2) (+ v1 v2)) (asarray -1 1) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [ ]:
bands = ['nir', 'red']

Define the input local STAC catalogs

In [ ]:
channel_inputs = [catalogs[0], 
                  catalogs[0]]
channel_inputs

In [ ]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [ ]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['aoi'] = aoi
params['lut'] = 'viridis'

In [ ]:
result = scombi(**params)

In [ ]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m

### Multitemporal RGB composite

In [ ]:
expressions = ['(interp v1 (asarray 0 10000) (asarray 0 1))', 
               '(interp v2 (asarray 0 10000) (asarray 0 1))',
               '(interp v3 (asarray 0 10000) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [ ]:
bands = ['red', 'green', 'blue']

Define the input local STAC catalogs

In [ ]:
channel_inputs = [catalogs[0], 
                  catalogs[1],
                  catalogs[2]]
channel_inputs

In [ ]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45' 

In [ ]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [ ]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['color'] = color
params['aoi'] = aoi


In [ ]:
result = scombi(**params)

In [ ]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m